# Golf Predictor 

## (0) Load Data

In [1]:
import pandas as pd
# dfRaw is raw dataset read from csv
dfRaw = pd.read_csv("../data_2015_thru_2019.csv")
dfRaw = dfRaw.drop(['RANK FOLLOWING YEAR'], axis=1) # remove rank following year col
dfRaw
distance_dict = {"A" : 1, "B" : 2, "C" : 3, "D" : 4, "E" : 5, "F" : 6}

### (0.1) Load Training Data from File

In [2]:
# --------- For Now we will be using 2015-2018 as TRAINING, 2019 as TESTING --------------
dfTrain = pd.read_csv("../training.csv")
# delete rank next year column
dfTrain = dfTrain.drop(['RANK FOLLOWING YEAR'], axis=1) # remove rank following year col
dfTrain

,PLAYER NAME,YEAR,ROUNDS,AVERAGE,TOTAL SG:ARG,MEASURED ROUNDS,ROUNDS_x,AVERAGE_x,SG:OTT,SG:APR,SG:ARG,AVERAGE_y,TOTAL SG:T,TOTAL SG:T2G,TOTAL SG:P,AVERAGE_x.1,TOTAL SG:PUTTING,AVERAGE_y.1,TOTAL SG:APP,RANK AS LABEL
0,Jason Day,2015,75,0.287,16.357,57,75,1.520,0.772,0.461,0.287,2.106,120.054,86.649,33.407,0.586,33.407,0.461,26.260,A
1,Dustin Johnson,2015,75,-0.212,-11.466,54,75,1.326,0.960,0.579,-0.212,1.455,78.552,71.623,6.932,0.128,6.932,0.579,31.252,A
2,Jordan Spieth,2015,91,0.471,31.054,66,91,1.583,0.494,0.618,0.471,2.154,142.131,104.470,37.665,0.571,37.665,0.618,40.776,A
3,Henrik Stenson,2015,63,0.082,3.604,44,63,1.774,0.448,1.244,0.082,2.210,97.245,78.041,19.205,0.436,19.205,1.244,54.732,A
4,Adam Scott,2015,51,-0.195,-6.238,32,51,0.601,0.682,0.114,-0.195,0.206,6.577,19.240,-12.662,-0.396,-12.662,0.114,3.659,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,Johnson Wagner,2018,75,0.226,12.456,55,75,0.092,-0.213,0.078,0.226,0.715,39.339,5.049,34.292,0.623,34.292,0.078,4.304,F
500,Scott Brown,2018,102,-0.175,-14.172,81,102,-0.710,-0.265,-0.270,-0.175,-0.661,-53.530,-57.498,3.967,0.049,3.967,-0.270,-21.856,F
501,Trey Mullinax,2018,75,-0.207,-12.199,59,75,-0.144,0.506,-0.442,-0.207,-0.072,-4.231,-8.472,4.243,0.072,4.243,-0.442,-26.106,F
502,Shawn Stefani,2018,76,0.013,0.809,60,76,0.141,-0.093,0.220,0.013,0.107,6.420,8.464,-2.045,-0.034,-2.045,0.220,13.227,F


### (0.2) Load Testing Data from File

In [3]:
dfTest = pd.read_csv("../testing.csv")
dfTest = dfTest.drop("RANK FOLLOWING YEAR", axis=1)
dfTest

,PLAYER NAME,YEAR,ROUNDS,AVERAGE,TOTAL SG:ARG,MEASURED ROUNDS,ROUNDS_x,AVERAGE_x,SG:OTT,SG:APR,SG:ARG,AVERAGE_y,TOTAL SG:T,TOTAL SG:T2G,TOTAL SG:P,AVERAGE_x.1,TOTAL SG:PUTTING,AVERAGE_y.1,TOTAL SG:APP,RANK AS LABEL
0,Dustin Johnson,2019,73,0.234,13.098,56,73,1.225,0.703,0.288,0.234,1.321,73.995,68.619,5.378,0.096,5.378,0.288,16.143,A
1,Jon Rahm,2019,75,0.073,3.863,53,75,1.170,0.692,0.405,0.073,1.505,79.788,61.994,17.793,0.336,17.793,0.405,21.449,A
2,Justin Thomas,2019,75,0.352,19.729,56,75,1.817,0.479,0.985,0.352,1.631,91.314,101.736,-10.419,-0.186,-10.419,0.985,55.176,A
3,Rory McIlroy,2019,72,0.297,16.939,57,72,2.126,1.195,0.633,0.297,2.551,145.379,121.159,24.221,0.425,24.221,0.633,36.096,A
4,Webb Simpson,2019,81,0.294,20.269,69,81,0.899,-0.007,0.613,0.294,1.507,104.007,62.051,41.957,0.608,41.957,0.613,42.285,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,Patton Kizzire,2019,76,-0.120,-6.337,53,76,-0.740,-0.489,-0.132,-0.120,-0.098,-5.192,-39.241,34.050,0.642,34.050,-0.132,-6.987,F
128,Peter Malnati,2019,87,0.097,7.362,76,87,-0.088,-0.410,0.225,0.097,0.379,28.774,-6.693,35.469,0.467,35.469,0.225,17.084,F
129,Bronson Burgoon,2019,59,-0.074,-3.481,47,59,-0.053,0.054,-0.033,-0.074,-0.344,-16.148,-2.469,-13.679,-0.291,-13.679,-0.033,-1.540,F
130,Chris Stroud,2019,70,0.035,1.808,52,70,-0.752,-0.557,-0.229,0.035,-0.729,-37.920,-39.088,1.165,0.022,1.165,-0.229,-11.918,F


### 0.2.2 K-Fold

In [4]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
kf.get_n_splits()
trainSets = []
testSets = []
for k in kf.split(dfRaw):
    trainSets.append(dfRaw.iloc[k[0]])
    testSets.append(dfRaw.iloc[k[1]])

### (0.3) Find Feature Names

In [5]:
featureNames = dfTrain.columns[3:-1] # Skip ID, name, year, rounds, and rank as label
nFeatures = featureNames.shape[0]
print("Number of features:", nFeatures)
featureNames

Number of features: 16


Index(['AVERAGE', 'TOTAL SG:ARG', 'MEASURED ROUNDS', 'ROUNDS_x', 'AVERAGE_x',
       'SG:OTT', 'SG:APR', 'SG:ARG', 'AVERAGE_y', 'TOTAL SG:T', 'TOTAL SG:T2G',
       'TOTAL SG:P', 'AVERAGE_x.1', 'TOTAL SG:PUTTING', 'AVERAGE_y.1',
       'TOTAL SG:APP'],
      dtype='object')

### (0.4) Select Training Instances (features and values) from the Training Data

In [6]:
trainingInstancesList = []
for train in trainSets:
    trainingInstances_x = []
    trainingInstances_y = []
    for instance in train.to_numpy():
        featureValues = list(instance[3:-1])
        label = instance[-1]
        trainingInstances_x.append(featureValues)
        trainingInstances_y.append(label)
    nTrainingInstances = len(trainingInstances_x)
    trainingInstancesList.append((trainingInstances_x, trainingInstances_y, nTrainingInstances))
    print("Number of training instances:", nTrainingInstances)

Number of training instances: 508
Number of training instances: 509
Number of training instances: 509
Number of training instances: 509
Number of training instances: 509


### (0.5) Select Test Instances from the Data

In [7]:
testInstancesList = []
for test in testSets:    
    testInstances_x = []
    testInstances_y = []
    for instance in test.to_numpy():
        featureValues = list(instance[3:-1])
        label = instance[-1]
        testInstances_x.append(featureValues)
        testInstances_y.append(label)
    nTrainingInstances = len(trainingInstances_x)
    print("Number of training instances:", nTrainingInstances)
    testInstancesList.append((testInstances_x, testInstances_y, nTrainingInstances))

Number of training instances: 509
Number of training instances: 509
Number of training instances: 509
Number of training instances: 509
Number of training instances: 509


### (0.6) MAE

In [8]:
def mae(testInstances, prediction):
    for i in range(len(prediction)):
        print('Predict: {}, Actual: {}'.format(prediction[i], testInstances[i]))

## 1. Support Vector Classification

### 1.1 Linear SVM

In [9]:
from sklearn.svm import LinearSVC
from sklearn import metrics
acc_sum = 0
f1micro_sum = 0
f1macro_sum = 0
MAE_sum = 0
for trainingInstances, testInstances in zip(trainingInstancesList, testInstancesList):
    trainingInstances_x = trainingInstances[0]
    trainingInstances_y = trainingInstances[1]
    testInstances_x = testInstances[0]
    testInstances_y = testInstances[1]
    clf = LinearSVC()
    clf.fit(trainingInstances_x, trainingInstances_y)
    y_pred = clf.predict(testInstances_x)
    
    # CALCULATIONS FOR MAE
    y_pred_as_distance = [distance_dict[a] for a in y_pred]
    y_ans_as_distance = [distance_dict[a] for a in testInstances_y]
    MAE_sum += metrics.mean_absolute_error(y_ans_as_distance, y_pred_as_distance)
    
    #print("Predicted:", y_pred)
    print("Accuracy:", metrics.accuracy_score(testInstances_y, y_pred))
    acc_sum += metrics.accuracy_score(testInstances_y, y_pred)
    print("F1 Micro:", metrics.f1_score(testInstances_y, y_pred, average='micro'))
    f1micro_sum += metrics.f1_score(testInstances_y, y_pred, average='micro')
    print("F1 Macro:", metrics.f1_score(testInstances_y, y_pred, average='macro'))
    f1macro_sum += metrics.f1_score(testInstances_y, y_pred, average='macro')
    y_pred_svm = y_pred
print("Overall MAE: ", MAE_sum / len(testSets))
print("Overall Accuracy: ", acc_sum / len(testSets))
print("Overall F1 Micro: ", f1micro_sum / len(testSets))
print("Overall F1 Macro: ", f1macro_sum / len(testSets))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


Accuracy: 0.3359375
F1 Micro: 0.3359375
F1 Macro: 0.2640344909739624
Accuracy: 0.2440944881889764
F1 Micro: 0.2440944881889764
F1 Macro: 0.17992694266806875


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


Accuracy: 0.3543307086614173
F1 Micro: 0.3543307086614173
F1 Macro: 0.21404621154335113


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


Accuracy: 0.30708661417322836
F1 Micro: 0.30708661417322836
F1 Macro: 0.2745427758585653
Accuracy: 0.16535433070866143
F1 Micro: 0.16535433070866143
F1 Macro: 0.061995133819951344
Overall MAE:  1.3348056102362205
Overall Accuracy:  0.2813607283464567
Overall F1 Micro:  0.2813607283464567
Overall F1 Macro:  0.19890911097277977


### 1.2 Kernel SVM  

Kernel is Radius Basis Function: Gaussian:  

In [10]:
from sklearn.svm import SVC
acc_sum = 0
f1micro_sum = 0
f1macro_sum = 0
MAE_sum = 0
for trainingInstances, testInstances in zip(trainingInstancesList, testInstancesList):
    trainingInstances_x = trainingInstances[0]
    trainingInstances_y = trainingInstances[1]
    testInstances_x = testInstances[0]
    testInstances_y = testInstances[1]
    clf = SVC()
    clf.fit(trainingInstances_x, trainingInstances_y)
    y_pred = clf.predict(testInstances_x)
    
    
    # CALCULATIONS FOR MAE
    y_pred_as_distance = [distance_dict[a] for a in y_pred]
    y_ans_as_distance = [distance_dict[a] for a in testInstances_y]
    MAE_sum += metrics.mean_absolute_error(y_ans_as_distance, y_pred_as_distance)
    
    
    #print("Predicted:", y_pred)
    print("Accuracy:", metrics.accuracy_score(testInstances_y, y_pred))
    acc_sum += metrics.accuracy_score(testInstances_y, y_pred)
    print("F1 Micro:", metrics.f1_score(testInstances_y, y_pred, average='micro'))
    f1micro_sum += metrics.f1_score(testInstances_y, y_pred, average='micro')
    print("F1 Macro:", metrics.f1_score(testInstances_y, y_pred, average='macro'))
    f1macro_sum += metrics.f1_score(testInstances_y, y_pred, average='macro')
    y_pred_svm = y_pred
print("Overall MAE: ", MAE_sum / len(testSets))
print("Overall Accuracy: ", acc_sum / len(testSets))
print("Overall F1 Micro: ", f1micro_sum / len(testSets))
print("Overall F1 Macro: ", f1macro_sum / len(testSets))

Accuracy: 0.359375
F1 Micro: 0.359375
F1 Macro: 0.2347613248322468
Accuracy: 0.3700787401574803
F1 Micro: 0.37007874015748027
F1 Macro: 0.279578452400594
Accuracy: 0.3700787401574803
F1 Micro: 0.37007874015748027
F1 Macro: 0.24412487154422638
Accuracy: 0.3937007874015748
F1 Micro: 0.39370078740157477
F1 Macro: 0.2923057157246667
Accuracy: 0.4330708661417323
F1 Micro: 0.4330708661417323
F1 Macro: 0.31790754591825066
Overall MAE:  0.9841781496062992
Overall Accuracy:  0.3852608267716535
Overall F1 Micro:  0.3852608267716535
Overall F1 Macro:  0.27373558208399684


Kernel is Quadratic kernel ("Degree-2 polynomial kernel"):

In [11]:
acc_sum = 0
f1micro_sum = 0
f1macro_sum = 0
MAE_sum = 0
for trainingInstances, testInstances in zip(trainingInstancesList, testInstancesList):
    trainingInstances_x = trainingInstances[0]
    trainingInstances_y = trainingInstances[1]
    testInstances_x = testInstances[0]
    testInstances_y = testInstances[1]
    clf = SVC(kernel='poly', degree=2)
    clf.fit(trainingInstances_x, trainingInstances_y)
    y_pred = clf.predict(testInstances_x)
    
    # CALCULATIONS FOR MAE
    y_pred_as_distance = [distance_dict[a] for a in y_pred]
    y_ans_as_distance = [distance_dict[a] for a in testInstances_y]
    MAE_sum += metrics.mean_absolute_error(y_ans_as_distance, y_pred_as_distance)
    
    #print("Predicted:", y_pred)
    print("Accuracy:", metrics.accuracy_score(testInstances_y, y_pred))
    acc_sum += metrics.accuracy_score(testInstances_y, y_pred)
    print("F1 Micro:", metrics.f1_score(testInstances_y, y_pred, average='micro'))
    f1micro_sum += metrics.f1_score(testInstances_y, y_pred, average='micro')
    print("F1 Macro:", metrics.f1_score(testInstances_y, y_pred, average='macro'))
    f1macro_sum += metrics.f1_score(testInstances_y, y_pred, average='macro')
    y_pred_svm = y_pred
print("Overall MAE: ", MAE_sum / len(testSets))
print("Overall Accuracy: ", acc_sum / len(testSets))
print("Overall F1 Micro: ", f1micro_sum / len(testSets))
print("Overall F1 Macro: ", f1macro_sum / len(testSets))

Accuracy: 0.3515625
F1 Micro: 0.3515625
F1 Macro: 0.24140211640211642
Accuracy: 0.3779527559055118
F1 Micro: 0.37795275590551175
F1 Macro: 0.2554452715743038
Accuracy: 0.36220472440944884
F1 Micro: 0.36220472440944884
F1 Macro: 0.23462213070998247
Accuracy: 0.4015748031496063
F1 Micro: 0.4015748031496063
F1 Macro: 0.2950817904977498
Accuracy: 0.4015748031496063
F1 Micro: 0.4015748031496063
F1 Macro: 0.28425925925925927
Overall MAE:  0.9810162401574803
Overall Accuracy:  0.37897391732283464
Overall F1 Micro:  0.37897391732283464
Overall F1 Macro:  0.26216211368868236


## 2. KNN

### Training KNN Classifiers

In [12]:
# KNN WITH n_neighbors=5
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
acc_sum = 0
f1micro_sum = 0
f1macro_sum = 0
MAE_sum = 0
for trainingInstances, testInstances in zip(trainingInstancesList, testInstancesList):
    trainingInstances_x = trainingInstances[0]
    trainingInstances_y = trainingInstances[1]
    testInstances_x = testInstances[0]
    testInstances_y = testInstances[1]
    clf = KNeighborsClassifier() # n_neighbors=5
    clf.fit(trainingInstances_x, trainingInstances_y)
    y_pred = clf.predict(testInstances_x)
    #print("Predicted:", y_pred)
    
    # CALCULATIONS FOR MAE
    y_pred_as_distance = [distance_dict[a] for a in y_pred]
    y_ans_as_distance = [distance_dict[a] for a in testInstances_y]
    MAE_sum += metrics.mean_absolute_error(y_ans_as_distance, y_pred_as_distance)
    
    print("Accuracy:", metrics.accuracy_score(testInstances_y, y_pred))
    acc_sum += metrics.accuracy_score(testInstances_y, y_pred)
    print("F1 Micro:", metrics.f1_score(testInstances_y, y_pred, average='micro'))
    f1micro_sum += metrics.f1_score(testInstances_y, y_pred, average='micro')
    print("F1 Macro:", metrics.f1_score(testInstances_y, y_pred, average='macro'))
    f1macro_sum += metrics.f1_score(testInstances_y, y_pred, average='macro')
    y_pred_svm = y_pred
print("Overall MAE: ", MAE_sum / len(testSets))
print("Overall Accuracy: ", acc_sum / len(testSets))
print("Overall F1 Micro: ", f1micro_sum / len(testSets))
print("Overall F1 Macro: ", f1macro_sum / len(testSets))

Accuracy: 0.359375
F1 Micro: 0.359375
F1 Macro: 0.2994049411393585
Accuracy: 0.3779527559055118
F1 Micro: 0.37795275590551175
F1 Macro: 0.3488184255884705
Accuracy: 0.4015748031496063
F1 Micro: 0.4015748031496063
F1 Macro: 0.36140942346501886
Accuracy: 0.3779527559055118
F1 Micro: 0.37795275590551175
F1 Macro: 0.3535038928871946
Accuracy: 0.33858267716535434
F1 Micro: 0.33858267716535434
F1 Macro: 0.30488824823689864
Overall MAE:  1.050369094488189
Overall Accuracy:  0.37108759842519684
Overall F1 Micro:  0.37108759842519684
Overall F1 Macro:  0.3336049862633882


In [13]:
# KNN WITH n_neighbors=3
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
acc_sum = 0
f1micro_sum = 0
f1macro_sum = 0
MAE_sum = 0
for trainingInstances, testInstances in zip(trainingInstancesList, testInstancesList):
    trainingInstances_x = trainingInstances[0]
    trainingInstances_y = trainingInstances[1]
    testInstances_x = testInstances[0]
    testInstances_y = testInstances[1]
    clf = KNeighborsClassifier(n_neighbors=3) # n_neighbors = 3
    clf.fit(trainingInstances_x, trainingInstances_y)
    y_pred = clf.predict(testInstances_x)
    #print("Predicted:", y_pred)

    # CALCULATIONS FOR MAE
    y_pred_as_distance = [distance_dict[a] for a in y_pred]
    y_ans_as_distance = [distance_dict[a] for a in testInstances_y]
    MAE_sum += metrics.mean_absolute_error(y_ans_as_distance, y_pred_as_distance)
    
    print("Accuracy:", metrics.accuracy_score(testInstances_y, y_pred))
    acc_sum += metrics.accuracy_score(testInstances_y, y_pred)
    print("F1 Micro:", metrics.f1_score(testInstances_y, y_pred, average='micro'))
    f1micro_sum += metrics.f1_score(testInstances_y, y_pred, average='micro')
    print("F1 Macro:", metrics.f1_score(testInstances_y, y_pred, average='macro'))
    f1macro_sum += metrics.f1_score(testInstances_y, y_pred, average='macro')
    y_pred_svm = y_pred
print("Overall MAE: ", MAE_sum / len(testSets))
print("Overall Accuracy: ", acc_sum / len(testSets))
print("Overall F1 Micro: ", f1micro_sum / len(testSets))
print("Overall F1 Macro: ", f1macro_sum / len(testSets))

Accuracy: 0.3125
F1 Micro: 0.3125
F1 Macro: 0.2734259649557016
Accuracy: 0.3543307086614173
F1 Micro: 0.3543307086614173
F1 Macro: 0.3410830999066293
Accuracy: 0.30708661417322836
F1 Micro: 0.30708661417322836
F1 Macro: 0.28332082551594745
Accuracy: 0.29133858267716534
F1 Micro: 0.29133858267716534
F1 Macro: 0.2975182556649522
Accuracy: 0.25984251968503935
F1 Micro: 0.25984251968503935
F1 Macro: 0.2466221777542853
Overall MAE:  1.25625
Overall Accuracy:  0.3050196850393701
Overall F1 Micro:  0.3050196850393701
Overall F1 Macro:  0.2883940647595032


## 4. Gaussian Naive Bayes

In [14]:
from sklearn.naive_bayes import GaussianNB
acc_sum = 0
f1micro_sum = 0
f1macro_sum = 0
MAE_sum = 0
for trainingInstances, testInstances in zip(trainingInstancesList, testInstancesList):
    trainingInstances_x = trainingInstances[0]
    trainingInstances_y = trainingInstances[1]
    testInstances_x = testInstances[0]
    testInstances_y = testInstances[1]
    clf = GaussianNB()
    clf.fit(trainingInstances_x, trainingInstances_y)
    y_pred = clf.predict(testInstances_x)
    #print("Predicted:"print("Accuracy:", metrics.accuracy_score(testInstances_y, y_pred))
    
    # CALCULATIONS FOR MAE
    y_pred_as_distance = [distance_dict[a] for a in y_pred]
    y_ans_as_distance = [distance_dict[a] for a in testInstances_y]
    MAE_sum += metrics.mean_absolute_error(y_ans_as_distance, y_pred_as_distance)
    
    print("Accuracy:", metrics.accuracy_score(testInstances_y, y_pred))
    acc_sum += metrics.accuracy_score(testInstances_y, y_pred)
    print("F1 Micro:", metrics.f1_score(testInstances_y, y_pred, average='micro'))
    f1micro_sum += metrics.f1_score(testInstances_y, y_pred, average='micro')
    print("F1 Macro:", metrics.f1_score(testInstances_y, y_pred, average='macro'))
    f1macro_sum += metrics.f1_score(testInstances_y, y_pred, average='macro')
    y_pred_svm = y_pred
print("Overall MAE: ", MAE_sum / len(testSets))
print("Overall Accuracy: ", acc_sum / len(testSets))
print("Overall F1 Micro: ", f1micro_sum / len(testSets))
print("Overall F1 Macro: ", f1macro_sum / len(testSets))

Accuracy: 0.390625
F1 Micro: 0.390625
F1 Macro: 0.34375385836493527
Accuracy: 0.3228346456692913
F1 Micro: 0.3228346456692913
F1 Macro: 0.29949546039239194
Accuracy: 0.36220472440944884
F1 Micro: 0.36220472440944884
F1 Macro: 0.3105789471380869
Accuracy: 0.4015748031496063
F1 Micro: 0.4015748031496063
F1 Macro: 0.41277586701315516
Accuracy: 0.2992125984251969
F1 Micro: 0.2992125984251969
F1 Macro: 0.2920152496125799
Overall MAE:  1.014271653543307
Overall Accuracy:  0.3552903543307087
Overall F1 Micro:  0.3552903543307087
Overall F1 Macro:  0.3317238765042298


# 5. Naive Solution - Random Guess

In [15]:
import random
acc_sum = 0
f1micro_sum = 0
f1macro_sum = 0
MAE_sum = 0
representative_sample = ["A", "A", "B", "B", "C", "C", "D", "D", "D", "D", "E", "E", "E", "E", "E", "E", "E", "E", "E",
                        "E", "F","F", "F", "F", "F", "F", "F", "F", "F", "F"]

for trainingInstances, testInstances in zip(trainingInstancesList, testInstancesList):
    trainingInstances_x = trainingInstances[0]
    trainingInstances_y = trainingInstances[1]
    testInstances_x = testInstances[0]
    testInstances_y = testInstances[1]
    y_pred = [random.sample(representative_sample, 1)[0] for y in testInstances_y]
    
    # CALCULATIONS FOR MAE
    y_pred_as_distance = [distance_dict[a] for a in y_pred]
    y_ans_as_distance = [distance_dict[a] for a in testInstances_y]
    MAE_sum += metrics.mean_absolute_error(y_ans_as_distance, y_pred_as_distance)
    
    print("Accuracy:", metrics.accuracy_score(testInstances_y, y_pred))
    acc_sum += metrics.accuracy_score(testInstances_y, y_pred)
    print("F1 Micro:", metrics.f1_score(testInstances_y, y_pred, average='micro'))
    f1micro_sum += metrics.f1_score(testInstances_y, y_pred, average='micro')
    print("F1 Macro:", metrics.f1_score(testInstances_y, y_pred, average='macro'))
    f1macro_sum += metrics.f1_score(testInstances_y, y_pred, average='macro')
    y_pred_svm = y_pred
print("Overall MAE: ", MAE_sum / len(testSets))
print("Overall Accuracy: ", acc_sum / len(testSets))
print("Overall F1 Micro: ", f1micro_sum / len(testSets))
print("Overall F1 Macro: ", f1macro_sum / len(testSets))

Accuracy: 0.265625
F1 Micro: 0.265625
F1 Macro: 0.17670034758642353
Accuracy: 0.16535433070866143
F1 Micro: 0.16535433070866143
F1 Macro: 0.11834247492849644
Accuracy: 0.2283464566929134
F1 Micro: 0.2283464566929134
F1 Macro: 0.17840993167080124
Accuracy: 0.2125984251968504
F1 Micro: 0.2125984251968504
F1 Macro: 0.16328229915473016
Accuracy: 0.2283464566929134
F1 Micro: 0.2283464566929134
F1 Macro: 0.1688551483536602
Overall MAE:  1.8589566929133858
Overall Accuracy:  0.22005413385826772
Overall F1 Micro:  0.22005413385826772
Overall F1 Macro:  0.16111804033882232


# 6. Naive Solution - Use Last Year's Rank

In [16]:
def convert_rank_to_class(raw_ans):
    ans = ""
    if raw_ans < 21:
        ans = "A"
    elif raw_ans < 41:
        ans = "B"
    elif raw_ans < 61:
        ans = "C"
    elif raw_ans < 101:
        ans = "D"
    elif raw_ans < 201:
        ans = "E"
    else:
        ans = "F"
    return ans


import random
acc_sum = 0
f1micro_sum = 0
f1macro_sum = 0
MAE_sum = 0

main_data = pd.read_csv("../data/combined.csv")
df_continuity = pd.read_csv("../owgr_2004-2019.csv")

y_pred = []
y_ans = []

# run predictions from 2004 to 2019 (using players in main_data)
for index, row in main_data[main_data['year'] >= 2004 & main_data['year'] <= 2019].iterrows():
    player = (df_continuity['name'] == row['name']) & (df_continuity['year'] == row['year'])
    player_ranking_data = df_continuity.loc[player]

    current_rank = player_ranking_data['current_rank'].values[0]
    rank_year_plus_two = player_ranking_data['rank_year_plus_two'].values[0]

    if rank_year_plus_two > 300:
        continue

    guess_instance = convert_rank_to_class(current_rank)
    ans_instance = convert_rank_to_class(rank_year_plus_two)

    y_pred.append(guess_instance)
    y_ans.append(ans_instance)

print(len(y_pred))
print(len(y_ans))

# CALCULATIONS FOR MAE
y_pred_as_distance = [distance_dict[a] for a in y_pred]
y_ans_as_distance = [distance_dict[a] for a in y_ans]
MAE_sum += metrics.mean_absolute_error(y_ans_as_distance, y_pred_as_distance)

print("Accuracy:", metrics.accuracy_score(y_ans, y_pred))
acc_sum += metrics.accuracy_score(y_ans, y_pred)
print("F1 Micro:", metrics.f1_score(y_ans, y_pred, average='micro'))
f1micro_sum += metrics.f1_score(y_ans, y_pred, average='micro')
print("F1 Macro:", metrics.f1_score(y_ans, y_pred, average='macro'))
f1macro_sum += metrics.f1_score(y_ans, y_pred, average='macro')

print("Overall MAE: ", MAE_sum / 1)
print("Overall Accuracy: ", acc_sum / 1)
print("Overall F1 Micro: ", f1micro_sum / 1)
print("Overall F1 Macro: ", f1macro_sum / 1)

1683
1683
Accuracy: 0.3196672608437314
F1 Micro: 0.3196672608437314
F1 Macro: 0.307310147545276
Overall MAE:  1.1354723707664884
Overall Accuracy:  0.3196672608437314
Overall F1 Micro:  0.3196672608437314
Overall F1 Macro:  0.307310147545276
